# Download Dataset from GitHub to Google Colaboratory

The custom dataset has been uploaded to the GitHub repository to clone the dataset and use it in this notebook.

In [ ]:
%cd /content
!git clone https://github.com/Intelligent-Systems-TCZR/HOG.git

# Show how HOG works
Input an image, and it will turn the image to grey and start to calculate the gradients.

In the end, it will show the image after implement the HOG feature.


In [ ]:
# Visualise HOG

import matplotlib.pyplot as plt
from skimage import io
from skimage.feature import hog
from skimage import data, color, exposure
from PIL import Image

# Input the image
img = io.imread(r"/content/HOG/sample/people.jpg")

# Turn the image to grey
image = color.rgb2gray(img)

# creating hog features
fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')
ax1.set_adjustable('box')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 0.02))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
ax1.set_adjustable('box')
plt.show()

#Train the SVM model

Here use two types of datasets to train the model, one is the positive samples (Human images), and the other is the negative samples (Non-Human images)

The dataset will split into 80% training dataset and 20% test dataset to see how well the model performed.

In [ ]:
# Training
# Importing the necessary modules:

from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage.io import imread
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import cv2
import os
import glob
from PIL import Image # This will be used to read/modify images (can be done via OpenCV too)
from numpy import *

# define parameters of HOG feature extraction
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3


# define path to images:

# This is the path of our positive input dataset
pos_im_path = r"/content/HOG/train/positive" 
# define the same for negatives
neg_im_path= r"/content/HOG/train/negative"

# read the image files:
pos_im_listing = os.listdir(pos_im_path) # it will read all the files in the positive image path (so all the required images)
neg_im_listing = os.listdir(neg_im_path)
num_pos_samples = size(pos_im_listing) # simply states the total no. of images
num_neg_samples = size(neg_im_listing)
print(num_pos_samples) # prints the number value of the no.of samples in positive dataset
print(num_neg_samples)
data= []
labels = []

# compute HOG features and label them:

for file in pos_im_listing: #this loop enables reading the files in the pos_im_listing variable one by one
    img = Image.open(pos_im_path + '/' + file) # open the file
    gray = img.convert('L') # convert the image into single channel i.e. RGB to grayscale
    # calculate HOG for positive features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    data.append(fd)
    labels.append(1)
    
# Same for the negative images
for file in neg_im_listing: #this loop enables reading the files in the neg_im_listing variable one by one
    img= Image.open(neg_im_path + '/' + file)
    gray= img.convert('L')
    # Now we calculate the HOG for negative features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True) 
    data.append(fd)
    labels.append(0)
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# Partitioning the data into training and testing splits, using 80%
# of the data for training and the remaining 20% for testing
print(" Constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(
	np.array(data), labels, test_size=0.20, random_state=42)
#%% Train the linear SVM
print(" Training Linear SVM classifier...")
model = LinearSVC()
model.fit(trainData, trainLabels)
#%% Evaluate the classifier
print(" Evaluating classifier on test data ...")
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))

# Save the model:
joblib.dump(model, 'model_name.npy')

#Run the HOG feature on Test Video and Image

In this section, we will input a video for it to detect the pedestrian on the road.
 
It will capture every frame of the video and display it whenever a human is detected. 

In [ ]:
# Testing
# Modified from OpenCV HOG person detector

import numpy as np
import cv2
import sys
from glob import glob
import itertools as it
from google.colab.patches import cv2_imshow

def inside(r, q):
    rx, ry, rw, rh = r
    qx, qy, qw, qh = q
    return rx > qx and ry > qy and rx + rw < qx + qw and ry + rh < qy + qh

def draw_detections(img, rects, thickness = 1):
    for x, y, w, h in rects:
        # the HOG detector returns slightly larger rectangles than the real objects.
        # so we slightly shrink the rectangles to get a nicer output.
        pad_w, pad_h = int(0.15*w), int(0.05*h)
        cv2.rectangle(img, (x+pad_w, y+pad_h), (x+w-pad_w, y+h-pad_h), (0, 0, 255), thickness)

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
''' the above code uses a pretrained SVM via HOG descriptors provided by the open cv database.
This database is limited to the training it has performed hence cannot be used in any other angle other than perp. to the centroid
Thus if you want to implement the HOG + SVM method, you'll have to train your own SVM with your own data'''
cap= cv2.VideoCapture("/content/HOG/test/video/video1.mp4")
# the above code uses the OpenCV library to capture video frames from the video (you can also input an image)

while True:
    #running an infinite loop so that the process is run real time.
    ret, img = cap.read() # reading the frames produced from the video in 'img' an then returning them using the 'ret' function.
    found, w = hog.detectMultiScale(img, winStride=(8,8), padding=(32,32), scale=1.05) # describing the parameters of HOG and returning them as a Human found function in 'found'
    found_filtered = [] #filtering the found human... to further improve visualisation (uses Gaussian filter for eradication of errors produced by luminescence.
    for ri, r in enumerate(found):
        for qi, q in enumerate(found):
            if ri != qi and inside(r, q):
                break
            else:
                found_filtered.append(r)
        draw_detections(img, found) # using the predefined bounding box to encapsulate the human detected within the bounding box.
        draw_detections(img, found_filtered, 3) # further filtering the box to improve visualisation.
    cv2_imshow(img) # finally showing the resulting image captured from the video.
    print("\n")